In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob
from keras.preprocessing import image                  
from tqdm import tqdm
from PIL import ImageFile  
from sklearn.model_selection import train_test_split
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint  
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.layers import BatchNormalization
from keras.regularizers import l1, l2
from keras.layers import GaussianNoise
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
import keras 
from keras.models import Model
from keras.applications import MobileNet

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    images = np.array(data['filenames'])
    images_targets = np_utils.to_categorical(np.array(data['target']), 3) # [1,0] is no_punch, [0,1] is right_punch
    return images, images_targets

#load training dataset
train_images, train_targets = load_dataset('Images/train') 
test_images, test_targets = load_dataset('Images/test')

print('There are %s total training images.\n' % len(train_images))
print('There are %s total testing images.\n' % len(test_images))

Using TensorFlow backend.


There are 1978 total training images.

There are 152 total testing images.



In [3]:
# Create functions to load the images into 4D numpy tensors, load data into tensors and create a training-validation split
# out of the training data.

# Functions to load images into 4d Numpy arrays.
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process and load the data for Keras
train_tensors = paths_to_tensor(train_images).astype('float32')/255
test_tensors = paths_to_tensor(test_images).astype('float32')/255

100%|██████████| 152/152 [00:23<00:00,  6.35it/s]


In [ ]:
# Visualize some of the training tensors
import matplotlib.pyplot as plt

for i in range (400,425):
    plt.imshow(train_tensors[i, :,:,:])
    plt.show()
    print(train_targets[i])

In [4]:

# Split the training data into training and validation sets.
train_tensors, valid_tensors, train_targets, valid_targets = train_test_split(train_tensors, train_targets, 
                                                                             test_size=0.2, 
                                                                             random_state=42)

In [5]:
# Set some adjustable variables outside the function and setup CNN compiling, fiting and testing function with early-stopping
# and model-checkpointing. Reload best weights and test model with best weights on testing set.

epochs = 1000
patience = 20
rmsprop_lowLR = optimizers.RMSprop(lr=0.00001)
optimizer= rmsprop_lowLR

def model_compile_fit(model):
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    print("Optimizer:", optimizer)
    print("Patience:", patience)
    
    checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
    
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto', 
          baseline=None, restore_best_weights=False)
    
    model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer,
          early_stopping], verbose=2)
    
    model.load_weights('saved_models/weights.best.from_scratch.hdf5')
    
    # get index of predicted action for each image in test set
    action_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

    # report test accuracy
    test_accuracy = 100*np.sum(np.array(action_predictions)==np.argmax(test_targets, axis=1))/len(2) #len(2) is len(diff actions)
    print('Test accuracy: %.4f%%' % test_accuracy)
              


In [6]:
#Same function as above but with ImageDataGenerator to allow image augmentation.

def augmented_data_model_compile_fit(model):
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    print("Optimizer:", optimizer)
    print("Patience:", patience)
    
    checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)
    
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto', 
          baseline=None, restore_best_weights=False)
    
    #Create and configure augmented image generator

    datagen =ImageDataGenerator(width_shift_range=0.1, # randomly shift images horizontally (10% of total width)
                                height_shift_range=0.1, # randomly shift images vertically (10% of total height)
                                zoom_range=0.2) # range for random zoom)
              
    
    #fit augmented image generator on data
    datagen.fit(train_tensors)
    
    batch_size = 20
    
    # train the model
    model.fit_generator(datagen.flow(train_tensors, train_targets, batch_size=batch_size),
                          steps_per_epoch=3*(train_tensors.shape[0]//batch_size),
                          epochs=epochs, verbose=2, callbacks=[checkpointer, early_stopping],
                         validation_data=(valid_tensors, valid_targets))
    
    # load the weights that generated the best validation accuracy
    model.load_weights('saved_models/weights.best.from_scratch.hdf5')
       
    # get index of predicted action for each image in test set
    action_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

    # report test accuracy
    test_accuracy = 100*np.sum(np.array(action_predictions)==np.argmax(test_targets, axis=1))/len(action_predictions) 
    print('Test accuracy: %.4f%%' % test_accuracy)

In [ ]:
patience= 15

aug_model9 = Sequential()
aug_model9.add(Conv2D(input_shape=(224,224,3), filters=16, kernel_size=(5,5), padding='same', activation='relu', strides=1))
aug_model9.add(MaxPooling2D(pool_size=(3,3)))
aug_model9.add(BatchNormalization())
aug_model9.add(Conv2D(input_shape=train_tensors.shape, filters=32, kernel_size=(4,4), padding='same', activation='relu', strides=1))
aug_model9.add(MaxPooling2D(pool_size=(2,2)))
aug_model9.add(BatchNormalization())
aug_model9.add(Conv2D(input_shape=train_tensors.shape, filters=64, kernel_size=(3,3), padding='same', activation='relu'))
aug_model9.add(MaxPooling2D(pool_size=(2,2), strides=1))
aug_model9.add(BatchNormalization())
aug_model9.add(Conv2D(input_shape=(224,224,3), filters=128, kernel_size=(3,3), padding='same', activation='relu'))
aug_model9.add(MaxPooling2D(pool_size=(2,2), strides=1))
aug_model9.add(BatchNormalization())
aug_model9.add(Conv2D(input_shape=(224,224,3), filters=256, kernel_size=(3,3), padding='same', activation='relu'))
aug_model9.add(MaxPooling2D(pool_size=(2,2), strides=1))
aug_model9.add(BatchNormalization())
aug_model9.add(Conv2D(input_shape=(224,224,3), filters=512, kernel_size=(3,3), padding='same', activation='relu'))
aug_model9.add(MaxPooling2D(pool_size=(2,2), strides=1))
aug_model9.add(BatchNormalization())
aug_model9.add(Conv2D(input_shape=(224,224,3), filters=1024, kernel_size=(3,3), padding='same', activation='relu'))
aug_model9.add(MaxPooling2D(pool_size=(2,2), strides=1))
aug_model9.add(BatchNormalization())
aug_model9.add(GlobalAveragePooling2D(data_format=None))
aug_model9.add(Dense(3, activation = 'softmax'))

aug_model9.summary()
augmented_data_model_compile_fit(aug_model9)

In [ ]:
# load the weights that generated the best validation accuracy
aug_model9.load_weights('saved_models/weights.best.from_scratch.hdf5')
       
# get index of predicted action for each image in test set
action_predictions = [np.argmax(aug_model9.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(action_predictions)==np.argmax(test_targets, axis=1))/len(action_predictions) 
print('Test accuracy: %.4f%%' % test_accuracy)

In [ ]:
# load the weights that generated the best validation accuracy
transfer_model.load_weights('saved_models/weights.best.transfer.hdf5')
       
# get index of predicted action for each image in test set
action_predictions = [np.argmax(transfer_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_bottlenecks]

# report test accuracy
test_accuracy = 100*np.sum(np.array(action_predictions)==np.argmax(test_targets, axis=1))/len(action_predictions) 
print('Test accuracy: %.4f%%' % test_accuracy)

In [7]:
###### Fine tuning MobileNet

# create the base pre-trained model
base_model = MobileNet(weights='imagenet', include_top=False)

base_model.summary()

E:\Anaconda3\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

_________________________________________________________________
conv_pw_9_relu (ReLU)        (None, None, None, 512)   0         
_________________________________________________________________
conv_dw_10 (DepthwiseConv2D) (None, None, None, 512)   4608      
_________________________________________________________________
conv_dw_10_bn (BatchNormaliz (None, None, None, 512)   2048      
_________________________________________________________________
conv_dw_10_relu (ReLU)       (None, None, None, 512)   0         
_________________________________________________________________
conv_pw_10 (Conv2D)          (None, None, None, 512)   262144    
_________________________________________________________________
conv_pw_10_bn (BatchNormaliz (None, None, None, 512)   2048      
_________________________________________________________________
conv_pw_10_relu (ReLU)       (None, None, None, 512)   0         
_________________________________________________________________
conv_dw_11

In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
fine_tuned_model = Model(inputs=base_model.input, outputs=predictions)
fine_tuned_model.summary()

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(fine_tuned_model.layers):
    print(i, layer.name)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

_________________________________________________________________
conv_pw_9_relu (ReLU)        (None, None, None, 512)   0         
_________________________________________________________________
conv_dw_10 (DepthwiseConv2D) (None, None, None, 512)   4608      
_________________________________________________________________
conv_dw_10_bn (BatchNormaliz (None, None, None, 512)   2048      
_________________________________________________________________
conv_dw_10_relu (ReLU)       (None, None, None, 512)   0         
_________________________________________________________________
conv_pw_10 (Conv2D)          (None, None, None, 512)   262144    
_________________________________________________________________
conv_pw_10_bn (BatchNormaliz (None, None, None, 512)   2048      
_________________________________________________________________
conv_pw_10_relu (ReLU)       (None, None, None, 512)   0         
_________________________________________________________________
conv_dw_11

In [9]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in fine_tuned_model.layers[:20]:
    layer.trainable = False
for layer in fine_tuned_model.layers[20:]:
    layer.trainable = True

In [10]:

# compile the model (should be done *after* setting layers to non-trainable)
fine_tuned_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto', 
          baseline=None, restore_best_weights=False)

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.fine_tuned_model.hdf5', 
                               verbose=1, save_best_only=True)

fine_tuned_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer, early_stopping], verbose=2)

Train on 1582 samples, validate on 396 samples
Epoch 1/1000


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv1/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@conv1_bn/cond/FusedBatchNorm/Switch"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 2, 2], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv1/kernel/read)]]
	 [[{{node metrics/acc/Mean/_1535}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7239_metrics/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [12]:
# load the weights that generated the best validation accuracy
fine_tuned_model.load_weights('saved_models/weights.best.fine_tuned_model.hdf5')
       
# get index of predicted action for each image in test set
action_predictions = [np.argmax(fine_tuned_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(action_predictions)==np.argmax(test_targets, axis=1))/len(action_predictions) 
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 99.3421%


In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
fine_tuned_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto', 
          baseline=None, restore_best_weights=False)

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.MobileNetv1.hdf5', 
                               verbose=1, save_best_only=True)

fine_tuned_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer, early_stopping], verbose=2)

In [6]:
from keras.applications import MobileNetV2

###### Fine tuning MobileNet

# create the base pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False)

base_model.summary()

E:\Anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:306: UserWarning: MobileNet shape is undefined. Weights for input shape(224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


9412608/9406464 [==============================] - ETA: 4: - ETA: 6: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 58 - ETA: 56 - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 55s - ETA: 55 - ETA: 53 - ETA: 54 - ETA: 56 - ETA: 53 - ETA: 51 - ETA: 49 - ETA: 46 - ETA: 43 - ETA: 41 - ETA: 44 - ETA: 46 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 38 - ETA: 39 - ETA: 38 - ETA: 39 - ETA: 34 - ETA: 35 - ETA: 36 - ETA: 36 - ETA: 36 - ETA: 37 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 30 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 26 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 25 - ETA: 25 - ETA: 24 - ETA: 24 - ETA: 24 - ETA: 25 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17

block_7_project_BN (BatchNormal (None, None, None, 6 256         block_7_project[0][0]            
__________________________________________________________________________________________________
block_7_add (Add)               (None, None, None, 6 0           block_6_project_BN[0][0]         
                                                                 block_7_project_BN[0][0]         
__________________________________________________________________________________________________
block_8_expand (Conv2D)         (None, None, None, 3 24576       block_7_add[0][0]                
__________________________________________________________________________________________________
block_8_expand_BN (BatchNormali (None, None, None, 3 1536        block_8_expand[0][0]             
__________________________________________________________________________________________________
block_8_expand_relu (ReLU)      (None, None, None, 3 0           block_8_expand_BN[0][0]          
__________

__________________________________________________________________________________________________
block_16_project (Conv2D)       (None, None, None, 3 307200      block_16_depthwise_relu[0][0]    
__________________________________________________________________________________________________
block_16_project_BN (BatchNorma (None, None, None, 3 1280        block_16_project[0][0]           
__________________________________________________________________________________________________
Conv_1 (Conv2D)                 (None, None, None, 1 409600      block_16_project_BN[0][0]        
__________________________________________________________________________________________________
Conv_1_bn (BatchNormalization)  (None, None, None, 1 5120        Conv_1[0][0]                     
__________________________________________________________________________________________________
out_relu (ReLU)                 (None, None, None, 1 0           Conv_1_bn[0][0]                  
Total para

In [13]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
fine_tuned_model = Model(inputs=base_model.input, outputs=predictions)
fine_tuned_model.summary()

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(fine_tuned_model.layers):
    print(i, layer.name)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, None, None, 3 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, None, None, 3 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

__________________________________________________________________________________________________
block_8_project (Conv2D)        (None, None, None, 6 24576       block_8_depthwise_relu[0][0]     
__________________________________________________________________________________________________
block_8_project_BN (BatchNormal (None, None, None, 6 256         block_8_project[0][0]            
__________________________________________________________________________________________________
block_8_add (Add)               (None, None, None, 6 0           block_7_add[0][0]                
                                                                 block_8_project_BN[0][0]         
__________________________________________________________________________________________________
block_9_expand (Conv2D)         (None, None, None, 3 24576       block_8_add[0][0]                
__________________________________________________________________________________________________
block_9_ex

Total params: 3,572,803
Trainable params: 3,538,691
Non-trainable params: 34,112
__________________________________________________________________________________________________
0 input_1
1 Conv1_pad
2 Conv1
3 bn_Conv1
4 Conv1_relu
5 expanded_conv_depthwise
6 expanded_conv_depthwise_BN
7 expanded_conv_depthwise_relu
8 expanded_conv_project
9 expanded_conv_project_BN
10 block_1_expand
11 block_1_expand_BN
12 block_1_expand_relu
13 block_1_pad
14 block_1_depthwise
15 block_1_depthwise_BN
16 block_1_depthwise_relu
17 block_1_project
18 block_1_project_BN
19 block_2_expand
20 block_2_expand_BN
21 block_2_expand_relu
22 block_2_depthwise
23 block_2_depthwise_BN
24 block_2_depthwise_relu
25 block_2_project
26 block_2_project_BN
27 block_2_add
28 block_3_expand
29 block_3_expand_BN
30 block_3_expand_relu
31 block_3_pad
32 block_3_depthwise
33 block_3_depthwise_BN
34 block_3_depthwise_relu
35 block_3_project
36 block_3_project_BN
37 block_4_expand
38 block_4_expand_BN
39 block_4_expand_relu


In [15]:
# compile the model (should be done *after* setting layers to non-trainable)
fine_tuned_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=0.0003), metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=patience, verbose=0, mode='auto', 
          baseline=None, restore_best_weights=False)

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.MobileNetV2.hdf5', 
                               verbose=1, save_best_only=True)

fine_tuned_model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer, early_stopping], verbose=2)

Train on 1582 samples, validate on 396 samples
Epoch 1/1000


InternalError: GPU sync failed